# Named entity recognition using MUSICBRAINZ API

Problem:
- Music personalities not identified in other databases

Approach
- Use MusicBrainz database and its API to search for entities missing
- Use musicbrainzngs python library to use the API
- Retrieve list of candidate names
- Use cosine similarity to reduce te number of candidate to the sematicaly closest ones
- Retrieve text describing the named entity

Data
- List of candidates as retrieved from MusicBrainz
- List of reduced candidates using cosine similarity
- Captured text describing entities

Requirements
- pip3 install musicbrainzngs
- pip3 install Levenshtein

In [2]:
import musicbrainzngs
from prettytable import PrettyTable
import json
import pandas as pd
import time
import os

In [3]:
musicbrainzngs.set_useragent("polifonia-ner", "0.1", "https://github.com/polifonia-project/meetups_pilot")

# result = musicbrainzngs.search_artists(artist="xx", type="group",
#                                        country="GB")

# search_artists
#     Search for artists and return a dict with an ‘artist-list’ key.
#     Available search fields: alias, area, arid, artist, artistaccent, begin, beginarea, comment, 
#     country, end, endarea, ended, gender, ipi, isni, primary_alias, sortname, tag, type
# Example, searcing an specific artist name
# result = musicbrainzngs.search_artists(artist="\"Sims Reeves\"", type="Person")
# Pring the results
# for artist in result['artist-list']:
#     # print(artist)
#     print(u"{id}: {name}".format(id=artist['id'], name=artist["name"]))
# Parse to dataframe for analysis
# df = pd.DataFrame.from_dict(pd.json_normalize(result['artist-list']), orient='columns')

In [9]:
# read file with entities not linked
entities_df = pd.read_csv('2023018_notLinkedProcess.csv')

i=1
# read each file in the list
for entity_name in entities_df.itertuples():
    print(str(entity_name.sent_n))
    # read each biography
    # sentences_df = pd.read_csv('../indexedSentences/'+file_name.file_name)
    # search_filter = '{}'.format(entity_name.mention)
    file_exists = os.path.isfile('results_json/'+str(i)+'_'+str(entity_name.sent_n)+'.csv')
    if not file_exists:
        # Clean text
        search_value = str(entity_name.pne)
        search_value = search_value.replace("Mr.", "")
        search_value = search_value.replace("Mr", "")
        search_value= search_value.replace("Mrs.", "")
        search_value= search_value.replace("Dr.", "").strip()
        search_value = "\"{}\"".format(search_value)
        # search_value = str(entity_name.mention)
    
        print(search_value)
        result = musicbrainzngs.search_artists(artist=search_value, type="Person",limit=100)
        result_df = pd.DataFrame.from_dict(pd.json_normalize(result['artist-list']), orient='columns')
        result_df.to_csv('results_json/'+str(i)+'_'+str(entity_name.sent_n)+'.csv',index=False)
        
        if i % 25 == 0:
            time.sleep(180)
            
        # if i == 25:
        #     time.sleep(180)
        # elif i == 50:
        #     time.sleep(180)
        # elif i == 75:
        #     time.sleep(180)
    i+=1
    

102529
"James Fitzgerald"
124652
"Robertson C.Clementi"
122276
"S. Nelson"
121611
"J. M. Murdie"
101661
"Richard Jeffries"
106396
"W. G. Marshall"
108230
"C. Gregory"
105937
"L. F. Pearse"
102699
"A. Ergmann"
99394
"H. C. Lunn"
110969
"J. D. Loper"
95261
"Josephine Grassini"
96865
"Maria Harris"
116898
"Carlo d’ Argentilly"
109978
"D. Farmer"
119008
"Colonel Stewart"
96184
"Smith"
120956
"Phillips"
100504
"A. S. Holloway"
105675
"Acomb"
105675
"Bowling"
102953
"R. R. Ambler"
115154
"Alexander Dallas"
120075
"Philippo D’Abadessa"
106722
"James Epps"
124658
"Linley"
110999
"Joseph Banister"
104547
"Lewis"
105669
"Hobart"
106258
"Albert Lowe"
122105
"J. Cohan"
122604
"Francis Fowke"
108216
"C. Morgan"
103988
"J. H. Pearson"
99536
"H. Ogle Moore"
124246
"J. C. Clifton"
101446
"Jessie Royd"
99890
"Van Heddeghem"
126166
"Wood"
127271
"Wood"
128376
"Wood"
124681
"C. Evans"
104349
"Arthur Wickes Cole"
110709
"A. Shaw"
102896
"B. P. WIllis"
109383
"J. L. Peters"
129904
"Quaranta"
131065
"Quaran

In [14]:
entities_df = pd.read_csv('2023018_notLinkedProcess.csv')

i=1
# read each file in the list
for entity_name in entities_df.itertuples():
    # print(str(entity_name.sent_n))
    # print(str(entity_name.mention))
    search_value = str(entity_name.pne)
    search_value = search_value.replace("Mr.", "")
    search_value = search_value.replace("Mr", "")
    search_value= search_value.replace("Mrs.", "")
    search_value= search_value.replace("Dr.", "").strip()
    # print(search_value)
    query_sentence = "name=='"+search_value+"'"
    df_result = pd.read_csv('results_json/'+str(i)+'_'+str(entity_name.sent_n)+'.csv')
    df_result = df_result.query(query_sentence)
    
    if len(df_result) > 0:
        print(str(i)+'_'+str(entity_name.sent_n) + ", " + str(entity_name.pne))
      
    i+=1

101661, Richard Jeffries
120956, Phillips
104547, Lewis
105669, Hobart
108216, C. Morgan
103988, J. H. Pearson
126166, Wood
127271, Wood
128376, Wood
109383, J. L. Peters
110495, Phillips
114002, Giardini


In [17]:
# Calculate Levensthain distance
import Levenshtein as Lev

In [35]:
entities_df = pd.read_csv('2023018_notLinkedProcess.csv')

i=1
# read each file in the list
for entity_name in entities_df.itertuples():
    # print(str(entity_name.sent_n))
    # print(str(entity_name.mention))
    search_value = str(entity_name.pne)
    search_value = search_value.replace("Mr.", "")
    search_value = search_value.replace("Mr", "")
    search_value= search_value.replace("Mrs.", "")
    search_value= search_value.replace("Dr.", "").strip()
    # print(search_value)
    query_sentence = "name=='"+search_value+"'"
    df_result = pd.read_csv('results_json/'+str(i)+'_'+str(entity_name.sent_n)+'.csv')
    

    df1 = df_result.query(query_sentence)
    if len(df1) > 0:
        print(str(i)+'_'+str(entity_name.sent_n) + ", " + str(entity_name.pne))
        df1.to_csv('candidates/'+str(i)+'_'+str(entity_name.sent_n)+'.csv',index=False)
    else:
        df_candidates = pd.DataFrame()
        list_c = []
        for candidate_entity in df_result.itertuples():
            distance = Lev.distance(entity_name.pne, candidate_entity.name) 
            print(str(entity_name.pne)+', '+str(candidate_entity.name) + ", " + str(distance))
            if distance <=4:
                # res = dict(zip(test_tup1, test_tup2))
                list_c.append(candidate_entity._asdict())
        # print(list_c)
        if len(list_c)  > 0:
            df_candidates = pd.DataFrame(list_c)
            df_candidates.to_csv('candidates/'+str(i)+'_'+str(entity_name.sent_n)+'.csv',index=False)
        
        # df_candidates = df_candidates.append(candidate_entity)
    
    i+=1

James Fitzgerald, Ella Fitzgerald, 5
James Fitzgerald, James Last, 9
James Fitzgerald, James Brown, 9
James Fitzgerald, James Horner, 8
James Fitzgerald, Johann Sebastian Bach, 17
James Fitzgerald, Etta James, 14
James Fitzgerald, James, 11
James Fitzgerald, James Taylor, 9
James Fitzgerald, Wolfgang Amadeus Mozart, 20
James Fitzgerald, Fitzgerald, 6
James Fitzgerald, Fitzgerald, 6
James Fitzgerald, Elmore James, 14
James Fitzgerald, George FitzGerald, 7
James Fitzgerald, Bruce Springsteen, 15
James Fitzgerald, James Newton Howard, 10
James Fitzgerald, Ludwig van Beethoven, 18
James Fitzgerald, Patrik Fitzgerald, 5
James Fitzgerald, Fitzgerald, 6
James Fitzgerald, Harry James, 13
James Fitzgerald, Scott Fitzgerald, 5
James Fitzgerald, James S.A. Corey, 10
James Fitzgerald, James Blunt, 10
James Fitzgerald, James Joyce, 9
James Fitzgerald, Dan FitzGerald, 5
James Fitzgerald, James Galway, 9
James Fitzgerald, Elvis Presley, 13
James Fitzgerald, David Fitzgerald, 4
James Fitzgerald, Anton